<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://drive.google.com/uc?export=view&id=1LqkEgbpZj8A99Y9T59eBp9fEIZbpg6P2" alt="Snowflake Snowpark Classroom" style="width: 800px">
</div>

# Classroom 1.5 - Column Operations Using Snowflake Snowpark Python API 

In this notebook, you will learn how to access column objects and perform operations using Snowflake Snowpark API for Python

## Learning Objectives

By the end of this classroom, you should be able to:
- Selecting Columns of a table or dataframe
- Understanding utility functions for constructing expressions with Columns
- Hands-on of methods for the most frequently used column transformations and operators

In [ ]:
# Let's get started with snowflake snowpark
from assets.config import connection_builder
session = connection_builder() 

## Section 1 - Understanding snowflake.snowpark.Column

snowflake.snowpark.Column represents a column or an expression in a DataFrame.

To access a Column object that refers a column in a DataFrame, you can:

- Use the column name.
- Use the functions.col() function.
- Use the DataFrame.col() method.
- Use the index operator [] on a dataframe object with a column name.
- Use the dot operator . on a dataframe object with a column name.

To create a Column object that represents a constant value, use [snowflake.snowpark.functions.lit()](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.functions.lit):

In [ ]:
from snowflake.snowpark.functions import col
# READING TABLE DATA
q_history = session.table('SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY')

# SELECTING QUERY_ID COLUMN USING COLUMN NAME
q_history.select('query_id').limit(10).show()

# SELECTING QUERY_ID USING DataFrame.col() METHOD
q_history.select(q_history.col('query_id')).limit(10).show()

# SELECTING QUERY_ID USING col() function
q_history.select(col('query_id')).limit(10).show()

# SELECTING QUERY_ID USING INDEX OPERATOR [] ON DataFrame
q_history.select(q_history['query_id']).limit(10).show()

# SELECTING QUERY_ID USING DOT OPERATOR . ON DataFrame
q_history.select(q_history.query_id).limit(10).show()

In [ ]:
from snowflake.snowpark.functions import lit
# CREATING COLUMN WITH CONSTANT VALUE
q_history.select(lit('column_with_constant_value')).limit(10).show()

## Section 2 - Understanding utility functions for constructing expressions with Columns

Column objects can be built with the operators, summarized by operator precedence, in the following table:

| Operator | Description |
| -- | -- |
| x[index] | Index operator to get an item out of a Snowflake ARRAY or OBJECT |
| ** | Power |
| -x, ~x | Unary minus, unary not |
| *, /, % | Multiply, divide, remainder | 
| +, - | Plus, minus | 
| & | And |
| &#124; | Or |
| ==, !=, <, <=, >, >= | Equal to, not equal to, less than, less than or equal to, greater than, greater than or equal to | 

In [ ]:
from snowflake.snowpark.types import StructField, StructType, IntegerType,StringType

schema = StructType([StructField('num1',IntegerType()),StructField('num2',IntegerType())])
df = session.create_dataframe([[20,34],[10,2],[10,10],[11,53]],schema=schema)

df.select(
    df.col('num1')**2,
    df.col('num1') + df.col('num2')*df.col('num1')-2/df.col('num1')%3,
).filter((df.col('num1') > 10)| (df.col('num2') < 10 ) ).show()

## Section 3 - Hands-on of methods for the most frequently used column transformations and operators


| Methods | Description |
| -- | -- |
| Column.as_(alias) / Column.alias(alias) | Returns a new renamed Column. |
| Column.asc() / Column.asc_nulls_first() / Column.asc_nulls_last() |  Returns a Column expression with values sorted in ascending order | 
| Column.astype(to) / Column.cast(to) / Column.try_cast(to) | Casts the value of the Column to the specified data type. |
| Column.between(lower_bound, upper_bound) | Between lower bound and upper bound. |
| Column.collate(collation_spec) | Returns a copy of the original Column with the specified collation_spec property, rather than the original collation specification property. |
| Column.desc() / Column.desc_nulls_first() / Column.desc_nulls_last() | Returns a Column expression with values sorted in descending order. |
| Column.endswith(other) | Returns true if this Column ends with another string. |
| Column.startswith(other) | Returns true if this Column starts with another string. |
| Column.eqNullSafe(other) / Column.equal_null(other) | Equal to. |
| Column.isNotNull() / Column.equal_nan() | Is NaN or Not Null. |
| Column.in_(*vals) / Column.isin(vals)  | Returns a conditional expression that you can pass to the DataFrame.filter() or where DataFrame.where() to perform the equivalent of a WHERE . |
| Column.like(pattern) | Allows case-sensitive matching of strings based on comparison with a pattern. |
| Column.substring(start_pos, length) / Column.substr(start_pos, length) | Returns a substring of this string column. |
| CaseExpr.when(condition, value) | Appends one more WHEN condition to the CASE expression.|
| CaseExpr.otherwise(value) | Sets the default result for this CASE expression. |

In [ ]:
from snowflake.snowpark.types import StructField, StructType, IntegerType,StringType,FloatType
from snowflake.snowpark.functions import when, upper, lit

df_schema = StructType([StructField('username',StringType()),StructField('age',IntegerType()),StructField('email_id',StringType()) ])
df_dataset = [['divyansh',24,'divyansh@abc.com'],['raghav',29,'raghav@abc.in'],['Aman',34,'aman@xyz.ca'],['Leo',22,'leo@abc.com'],['raghav',29,'raghav@abc.in'],['raghav',29,'raghav@abc.in'],['Aman',34,'aman@qwe.ru'],['  diño',41,'dino@pog.uk'],['Edward',33,None]]
df = session.createDataFrame(df_dataset,df_schema)
df.write.save_as_table('test_table',mode='overwrite',table_type='temporary')

test_table = session.table('test_table')
# Hands-on For CASE (When-Otherwise) expression using functions.when()
print('----------------- CASE Expression Example on Age Column -----------------')
test_table.select(when(test_table.age.is_null(),lit('10'))
          .when(test_table.age > 30,test_table.age*10)
          .otherwise(test_table.age)).show()


# Hands-on For Alias Function .alias() / .as_() for renaming the Column Name
print('----------------- Alias Function Example on Age Column -----------------')
test_table.select(test_table.age.alias('alias_for_age'),
                  test_table.email_id.as_('Alias_for_email')).show()

# Hands-on For Ascending Function .asc() / .asc_nulls_first() / .asc_nulls_last() for sorting the Column
print('----------------- Ascending Example on Username Column -----------------')
test_table.sort(test_table.age.asc_nulls_first()).show()
test_table.sort(test_table.age.asc()).show()
test_table.sort(test_table.age.asc_nulls_last()).show()

# Hands-on For Descending Function .desc() / .desc_nulls_first() / .desc_nulls_last() for sorting the Column
print('----------------- Descending Example on Username Column -----------------')
test_table.sort(test_table.age.desc_nulls_first()).show()
test_table.sort(test_table.age.desc()).show()
test_table.sort(test_table.age.desc_nulls_last()).show()

# Hands-on For Type Cast Function .astype() / cast() / try_cast() for data type conversion for the Column
print('----------------- AsType/ Cast/ Try_Cast Example on Age Column -----------------')
test_table.select(test_table.age.astype(FloatType()).as_('age1'),
                  test_table.age.cast(FloatType()),
                  test_table.username.try_cast(IntegerType())).show()

# Hands-on For Between Function .between() for getting the bool based on lower bound and upper bound for the Column
print('----------------- Between Example on Age Column -----------------')
test_table.filter((test_table.age.between(25,30))).show()

# Hands-on for In fuction .in_() / .isin()
print('----------------- In Example on Age and Username Column -----------------')
test_table.filter((test_table.age.in_(22,34,55)) & 
                  (test_table.username.isin('divyansh','Leo'))).show()


# Hands-on For Collate Function .collate() for returning with the specified collation_spec property of the Column
print('----------------- Collage Example on Username Column -----------------')
test_table.select(test_table.username,
                  test_table.username.collate('en_US-trim')).show()

# Hands-on For Startswith Function .startswith() .  A Column or a str that is used to check if this column starts with it. A str will be interpreted as a literal value instead of a column name.
print('----------------- startswith() Example on email_id Column -----------------')
test_table.filter(test_table.email_id.startswith('d')).show()

# Hands-on For Endswith Function .endswith() .  A Column or a str that is used to check if this column ends with it. A str will be interpreted as a literal value instead of a column name.
print('----------------- endswith() Example on email_id Column -----------------')
test_table.filter(test_table.email_id.endswith('ca')).show()

# Hands-on For Substring Function .substring() / .substr()
print('----------------- substring() / substr() Example on email_id and username Column -----------------')
test_table.select(test_table.username.substring(1,4),
                  test_table.email_id.substr(-6,3)).show()

# Hands-on to compare columns against null values
print('----------------- eqNullSafe()/equal_null() Example on email_id Column -----------------')
test_table.filter((test_table.email_id.eqNullSafe(df.email_id))).show()
test_table.filter((test_table.email_id.equal_null(df.email_id))).show()

# Hands-on to Identify if the value is null or not
print('----------------- equal_nan() Example on email_id Column -----------------')
test_table.filter((test_table.email_id.equal_nan())).show()
print('----------------- isNotNull() Example on email_id Column -----------------')
test_table.filter((test_table.email_id.isNotNull())).show()

# Hands-on For Like Function. Allows case-sensitive matching of strings based on comparison with a pattern.
test_table.filter(test_table.username.like("divya%")).show()

